In [2]:
import numpy as np
import pandas as pd
import sys
pd.set_option('display.width', 5000) 
pd.set_option('display.max_columns', 60)

from matplotlib import pyplot as plt

# Handy functions
sys.path.append("/app/tools")
from bcatools import *
from tqdm import tqdm_notebook
tqdm_notebook().pandas()

In [3]:
df_listings = pd.read_csv('listings.csv')

In [4]:
df_listings

,listing_id,Agent,sold_price,street_number,street_name,suburb,postcode,state,Municipality,Lister,property_type,bedrooms,NumberBathrooms,bathrooms,toilets,buildingsize,landsize,garages,carports,AirConditioning,AlarmSystem,TennisCourt,sold_date,year_built,longitude,latitude
0,108865901,GRMNAA,590000,71,Harold Keys Drive,Narre Warren South,3805,Vic,NaN,Andrea O'Connor 0439 988 714,Residential,4,0,2,2.0,19.052121,5.000000e+02,2.0,0.0,0,0,0,2018-11-01,NaN,145.289728,-38.026351
1,109102846,DDBEUB,375000,6,Bromley Street,Wyndham Vale,3024,Vic,NaN,Samantha McCarthy 0431 712 899,Residential,3,0,1,NaN,1.399308,2.590000e+02,0.0,1.0,0,0,0,2018-02-22,NaN,144.624596,-37.887191
2,110383117,GRMNAA,600000,7,Cottonwood Court,Narre Warren,3805,Vic,NaN,Gavin Coyne 0417 309 650,Residential,4,0,2,2.0,14.961835,7.750000e+02,2.0,0.0,0,0,0,2018-11-03,NaN,145.317704,-38.036736
3,113313443,XRWBUE,700000,3,Bowen Court,Bundoora,3083,Vic,NaN,Andrew Mizzi 0410 304 000,Residential,3,0,1,1.0,15.844476,6.800000e+02,2.0,0.0,0,0,0,2018-09-10,NaN,145.065538,-37.695670
4,116703895,REXROM,1100000,22,Heddle Road,Lancefield,3435,Vic,Macedon Ranges,NaN,Residential,5,0,2,3.0,3.982647,4.125000e+03,2.0,NaN,0,0,0,2018-02-26,NaN,144.723833,-37.255367
5,117378659,GHBDWE,480000,1,Barnes Way,Koo Wee Rup,3981,Vic,NaN,NaN,Residential,3,0,2,2.0,2.152782,6.160000e+02,2.0,0.0,0,0,0,2018-02-22,NaN,145.485935,-38.204549
6,119048447,GRMNAA,440000,5/1,Young Road,Hallam,3803,Vic,NaN,NaN,Residential,3,0,1,2.0,14.154542,1.420000e+02,1.0,0.0,0,0,0,2018-09-04,NaN,145.274587,-38.003366
7,119152095,XRWBUE,1200000,111,Edmund Rice Parade,Bundoora,3083,Vic,NaN,NaN,Residential,4,0,3,4.0,36.999866,7.050000e+02,2.0,0.0,0,0,0,2018-02-12,NaN,145.076035,-37.692684
8,119609603,ABCCRQ,360000,26,Wainewright Avenue,Mooroolbark,3138,Vic,NaN,NaN,Residential,3,0,2,0.0,1.722226,4.020000e+02,0.0,1.0,0,0,0,2018-09-05,NaN,145.324078,-37.769074
9,121139734,GHBDWE,495000,23,Carey Crescent,Pakenham,3810,Vic,NaN,NaN,Residential,3,0,2,2.0,18.836843,5.760000e+02,2.0,0.0,0,0,0,2018-08-22,NaN,145.499887,-38.056897


In [ ]:
df_listings[:5]


geo1 = (-37.852953, 144.724428)
geo2 = (-37.763273, 145.019996)

distance = haversine_distance(geo1, geo2)
brng = bearing(geo1, geo2)

print("The distance between the two points is {:.3f}Km, and the direction of B from A is {:.2f} degrees from North.".format(distance, brng))

mel_cbd_loc = (-37.8143349, 144.9624329)

def dist_from_cbd(loc):
    # FILL IN THE BLANK HERE
    dist = haversine_distance(mel_cbd_loc, loc)
    return dist


# Now we can apply this function to every row of our pandas dataframe, selecting only the `latitude` and `longitude` columns and save the result in a new column called `dist from cbd`.

# In[7]:


df_listings['dist_from_cbd'] = df_listings[['latitude','longitude']].apply(dist_from_cbd, axis=1)


# Repeat this for the bearing to add a column called `dir_from_cbd`

# In[ ]:





# In[8]:


# REMOVE THIS CODE
def dir_from_cbd(loc):
    # FILL IN THE BLANK HERE
    brng = bearing(mel_cbd_loc, loc)
    return brng

df_listings['dir_from_cbd'] = df_listings[['latitude','longitude']].apply(dir_from_cbd, axis=1)


# # Find the closest train station
# 
# Now we wish to find which train station is closest to each property.

# ### Load the station dataset
# The file is `'../data/stations.csv'`

# In[9]:


# REMOVE THIS CODE
df_stations = pd.read_csv('/app/data/stations.csv', index_col="station_id")


# In[10]:


df_stations[:5]


# In[ ]:





# We want to add two columns to our `df_listings`:
#    1. `station_id`, which is the `station_id`, which is the index of the station in the `df_stations` dataframe.
#    2. `dist_to_station`, which is the distance to that station.
#    
# You can do this anyway you like. 

# In[11]:


station_locs = df_stations[['latitude', 'longitude']].values
# An array of [[lat, lon],...] for all the stations


def closest_station(loc):
    '''
    Take a sing pair (lat, lon) and returns the (idx, distance) of the closest station in the list `station_locs`
    '''
    distances = np.apply_along_axis(lambda x: haversine_distance(loc,x), 1, station_locs)
    idx = np.argmin(distances)
    dist = distances[idx]
    return idx, dist


listing_locs = df_listings[['latitude', 'longitude']].values
# Array of [[lat, lon]] for the listings

# map over `listing_locs` and apply func `closest_station`
closest_stations = np.apply_along_axis(closest_station, 1, listing_locs)


# In[ ]:


ids, distance = zip(*closest_stations)


# In[ ]:


df_listings["station_id"] = ids
df_listings["dist_to_station"] = distance


# ### Export the dataset to csv

# In[ ]:


export_loc = '/app/data/processed/sold_listings_with_station.csv'
df_listings.to_csv(export_loc, index=False)


# In[ ]:




